In [1]:
import csv
import pandas as pd
from pygenprop.flat_file_parser import parse_genome_property_file
from pygenprop.results import assign_result_from_child_assignment_results, assign_property_result_from_required_steps, assign_results_to_property_and_children,GenomePropertiesResults
from pygenprop.assignment_file_parser import parse_genome_property_longform_file

In [2]:
interpro_ids = []
with open('/Users/lee/Google_Drive/Neufeld_Lab/Projects/ELA_GSB_test_data_for_Lee/Jackson_InterProScan_GenProp_Results/Updated_Fall_2018/interproscan_results/C_chlorochromatii_CaD3.tsv') as inteproscan_file:
    tsv_reader = csv.reader(inteproscan_file, delimiter='\t')
    
    for row in tsv_reader:
        hit_interpro_member_id = row[4]
        interpro_ids.append(hit_interpro_member_id)

In [3]:
unique_interpro_ids = set(interpro_ids)

In [4]:
with open('/Users/lee/Dropbox/RandD/Repositories/genome-properties/flatfiles/genomeProperties.txt') as genome_properties_file:
    genprop_tree = parse_genome_property_file(genome_properties_file)

In [5]:
step_table = {}
for leaf in genprop_tree:
    for step in leaf.steps:
        step_functional_element_assignments = []
        for functional_element in step.functional_elements:
            functional_element_evidence_assignments = []
            if functional_element.evidence:
                for evidence in functional_element.evidence:
                    if not evidence.has_genome_property:
                        if unique_interpro_ids.isdisjoint(set(evidence.evidence_identifiers)):
                            evidence_result = 'NO'
                        else:
                            evidence_result = 'YES'
                            
                        if evidence.sufficient:
                            if evidence_result == 'YES':
                                functional_element_evidence_assignments = ['YES']
                                break
                            else:
                                functional_element_evidence_assignments.append(evidence_result)
                        else:
                            functional_element_evidence_assignments.append(evidence_result)   
                    else:
                        break
            else:
                functional_element_evidence_assignments.append('NO')
                       
            if 'YES' in functional_element_evidence_assignments:
                step_functional_element_assignments.append('YES') 
        
        if 'YES' in step_functional_element_assignments:
            step_result = 'YES'

            current_genome_property_step_results = step_table.get(leaf.id)

            if current_genome_property_step_results:
                step_table[leaf.id][step.number] = step_result
            else:
                step_table[leaf.id] = {step.number: step_result} 

In [6]:
property_results={}
assign_results_to_property_and_children(property_results, step_table, genprop_tree.root)

'PARTIAL'

In [7]:
property_result_dict = {}
for property_id, assignment in property_results.items():
    supported_steps = [step_number for step_number, step_result in step_table[property_id].items() if step_result in ['YES', 'PARTIAL']]
    property_result_dict[property_id] = {'result': assignment,
                                         'supported_steps': supported_steps}
property_result_dict['sample_name'] = 'C_chlorochromatii_CaD3_InterProScanTSV'

In [8]:
interpro_file_results = GenomePropertiesResults(property_result_dict, genome_properties_tree=genprop_tree)

In [9]:
with open('/Users/lee/Google_Drive/Neufeld_Lab/Projects/ELA_GSB_test_data_for_Lee/Jackson_InterProScan_GenProp_Results/Updated_Fall_2018/genome_properties/C_chlorochromatii_CaD3') as assignment_file:
    file_assignment_results = parse_genome_property_longform_file(assignment_file)
    genome_properties_file_result = GenomePropertiesResults(file_assignment_results, genome_properties_tree=genprop_tree)

In [10]:
property_comparison_result = pd.concat([interpro_file_results.property_results, genome_properties_file_result.property_results], axis=1, sort=False)

In [11]:
step_comparison_result = pd.concat([interpro_file_results.step_results, genome_properties_file_result.step_results], axis=1, sort=False)

In [12]:
property_transposed = property_comparison_result.transpose()

In [13]:
property_transposed[[col for col in property_transposed if not property_transposed[col].nunique()==1]].transpose()

,C_chlorochromatii_CaD3_InterProScanTSV,C_chlorochromatii_CaD3
GenProp0305,NO,YES
GenProp0617,NO,PARTIAL
GenProp0616,NO,YES
GenProp0218,NO,PARTIAL
GenProp0645,NO,YES
GenProp0630,NO,PARTIAL
GenProp0033,PARTIAL,YES


In [14]:
step_transposed = step_comparison_result.transpose()

In [15]:
filtered_steps = step_transposed[[col for col in step_transposed if not step_transposed[col].nunique()==1]].transpose()

In [16]:
filtered_steps

,,C_chlorochromatii_CaD3_InterProScanTSV,C_chlorochromatii_CaD3
Genome_Property_ID,Step_Number,,
GenProp0033,4,NO,YES
GenProp0063,15,NO,YES
GenProp0126,15,NO,YES
GenProp0305,1,NO,YES
GenProp0616,1,NO,YES
GenProp0645,1,NO,YES


In [17]:
genprop_tree['GenProp0033']

GenProp0033, Type: METAPATH, Name: TCA cycle, Thresh: 5, References: False, Databases: True, Steps: True, Parents: True, Children: True, Public: False